In [ ]:
!pip install kora -q

In [ ]:
import torch
from torch.utils.data import Dataset
import os
import pickle
import argparse
from collections import defaultdict
import pandas as pd
import numpy as np
import gzip
import shutil
import spacy 
import pandas as pd 
from tqdm import tqdm
from collections import Counter
import kora.install.rdkit
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Draw, MolFromSmiles, MolToSmiles
import math
import torch.nn as nn
from rdkit import Chem
from rdkit import rdBase

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
import math
import torch.nn as nn
from rdkit import Chem
from rdkit import rdBase
rdBase.DisableLog('rdApp.*')

def split(sm):
    sm_couples = []
    i = 0
    while i < len(sm):
        if i == len(sm) - 1:
            sm_couples.append(sm[i])
            i += 1
        elif sm[i] == '%':
            sm_couples.append(sm[i:i+3])
            i += 3
        elif sm[i:i+2] in set(['Cl', 'Ca', 'Cu', 'Br', 'Be', 'Ba', 'Bi', 'Si', 'Se', 'Sr', 'Na', 'Ni', 'Rb', 'Ra', 'Xe', 'Li', 'Al', ' As', 'Ag', 'Au',\
                            'Mg', 'Mn', 'Te', 'Zn', 'si', 'se', 'te', 'He', 'Kr', 'Fe', '+2', '+3', '+4', '-2', '-3', '-4']):
            sm_couples.append(sm[i:i+2])
            i += 2
        else:
            sm_couples.append(sm[i])
            i += 1
    return ' '.join(sm_couples)

In [ ]:
with gzip.open('/content/drive/MyDrive/data/chembl_24_chemreps.txt.gz', 'rb') as f_in:
    with open('file.txt', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

df = pd.read_csv('file.txt', sep = '\t')

smiles  = df['canonical_smiles'].values

pro_sms = []
for sm in smiles:
    sm = ' '.join(list(sm))
    before = ['C l -', 'C l', 'O -', 'N +', 'n +', 'B r -', 'B r', 'N a +', 'N a', 'I -', 'S i']
    after = ['Cl-', 'Cl', 'O-', 'N+', 'n+', 'Br-', 'Br', 'Na+', 'Na', 'I-', 'Si']
    for b,a in zip(before, after):
        sm = sm.replace(b, a)
    pro_sms.append(sm)
df['processed_smiles'] = pro_sms

to_drop = []
for i, sm in enumerate(smiles):
    if len(sm) > 100:
        to_drop.append(i)
    if df['chembl_id'][i]=='CHEMBL1201364':
        to_drop.append(i)

df_dropped = df.drop(to_drop)
df_dropped = df_dropped.drop(['standard_inchi', 'standard_inchi_key'], axis=1)

df_dropped.to_csv('/content/drive/MyDrive/data/chembl_24_dropped.csv')
df_dropped = pd.read_csv('/content/drive/MyDrive/data/chembl_24_dropped.csv')

with open('/content/drive/MyDrive/data/chembl_24_corpus.txt', 'w') as f:
    for sm in tqdm(df_dropped['canonical_smiles']):
        f.write(split(sm) + '\n')

with open('/content/drive/MyDrive/data/chembl_24_corpus.txt', "r", encoding='utf-8') as f:
    vocab = Vocab(f, min_freq=500)
vocab.save_vocab('/content/drive/MyDrive/data/vocab24.pkl')